In [1]:
from common.evaluate import make_vis_env, test_env

from flow.multiagent_envs import MultiWaveAttenuationMergePOEnv
from flow.scenarios import MergeScenario
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from ray.tune import run_experiments

import gym, ray
from ray.rllib.agents.ppo import PPOAgent, DEFAULT_CONFIG
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

benchmark_name = 'multi_merge'

In [2]:
alg_run = "PPO"
num_rollouts = 60
num_cpus = 63
gae_lambda = 0.97
step_size = 5e-4

In [3]:
ray.init(num_cpus=num_cpus, include_webui=False, ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-18_22-06-04_113/logs.
Waiting for redis server at 127.0.0.1:22417 to respond...
Waiting for redis server at 127.0.0.1:20317 to respond...
Starting the Plasma object store with 20.0 GB memory using /dev/shm.


{'node_ip_address': '10.138.0.2',
 'object_store_addresses': ['/tmp/ray/session_2019-04-18_22-06-04_113/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-04-18_22-06-04_113/sockets/raylet'],
 'redis_address': '10.138.0.2:22417',
 'webui_url': ''}

In [4]:
benchmark = __import__(
    "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.flow_params
horizon = flow_params['env'].horizon

def make_env(create_env):
    def _thunk():
        env = create_env()
        return env
    return _thunk

create_env, env_name = make_create_env(params=flow_params, version=0)

In [5]:
config = DEFAULT_CONFIG.copy()

In [6]:
config["num_workers"] = min(num_cpus, num_rollouts) - 1
config["train_batch_size"] = horizon * num_rollouts
config["use_gae"] = True
config["horizon"] = horizon
config["lambda"] = gae_lambda
config["lr"] = step_size
config["vf_clip_param"] = 1e6
config["num_sgd_iter"] = 10
config['clip_actions'] = False  # FIXME(ev) temporary ray bug
config["model"]["fcnet_hiddens"] = [100, 50, 25]
config["observation_filter"] = "NoFilter"

# save the flow params for replay
flow_json = json.dumps(
    flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
config['env_config']['flow_params'] = flow_json
config['env_config']['run'] = alg_run

In [7]:
# Register as rllib env
register_env(env_name, create_env)

In [8]:
exp_tag = {
    "run": alg_run,
    "env": env_name,
    "config": {
        **config
    },
    "checkpoint_freq": 25,
    "max_failures": 999,
    "stop": {
        "training_iteration": 500
    },
    "num_samples": 1,
}

In [9]:
trials = run_experiments({
        flow_params["exp_tag"]: exp_tag
    })

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/63 CPUs, 0/0 GPUs
Memory usage on this node: 3.0/94.8 GB

Created LogSyncer for /headless/ray_results/multi_merge/PPO_MultiWaveAttenuationMergePOEnv-v0_0_2019-04-18_22-06-061w6z291m -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 3.3/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-18_22-07-17
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.20000001788139343
      cur_lr: 0.0005000000237487257
      entropy: 1.4259556531906128
      kl: 0.0009673548629507422

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 18.1/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING [pid=361], 156 s, 6 iter, 429678 ts, 438 rew

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-18_22-09-34
  done: false
  episode_len_mean: 750.0
  episode_reward_max: 804.927290589831
  episode_reward_mean: 522.8962128665526
  episode_reward_min: 229.04169320792337
  episodes_this_iter: 21
  episodes_total: 126
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.050000011920928955
      cur_lr: 0.0005000000237487257
      entropy: 1.3389524221420288
      kl: 0.00568394735455513
      policy_loss: -0.0016918210312724113
      total_loss: 4.6704840660095215
      vf_explained_var: 0.2818896472454071
      vf_loss: 4.671890735626221
    grad_ti

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 18.1/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING [pid=361], 287 s, 12 iter, 833051 ts, 712 rew

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-18_22-11-44
  done: false
  episode_len_mean: 750.0
  episode_reward_max: 867.2541512893827
  episode_reward_mean: 712.5467401418788
  episode_reward_min: 627.6192255625829
  episodes_this_iter: 15
  episodes_total: 257
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.006250000558793545
      cur_lr: 0.0005000000237487257
      entropy: 1.3412318229675293
      kl: 0.003936938010156155
      policy_loss: -0.000680316414218396
      total_loss: 5.847048759460449
      vf_explained_var: 0.29600462317466736
      vf_loss: 5.8477044105529785
    grad_

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 18.2/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING [pid=361], 411 s, 18 iter, 1233112 ts, 710 rew

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-18_22-13-48
  done: false
  episode_len_mean: 750.0
  episode_reward_max: 859.8649373384592
  episode_reward_mean: 713.9495546168265
  episode_reward_min: 631.9345002521995
  episodes_this_iter: 23
  episodes_total: 393
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.0007812500698491931
      cur_lr: 0.0005000000237487257
      entropy: 1.2977361679077148
      kl: 0.008466006256639957
      policy_loss: -0.0017513647908344865
      total_loss: 8.044719696044922
      vf_explained_var: 0.17817899584770203
      vf_loss: 8.046463966369629
    gra

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 18.3/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING [pid=361], 541 s, 24 iter, 1631942 ts, 716 rew

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-18_22-15-58
  done: false
  episode_len_mean: 750.0
  episode_reward_max: 868.5971910197102
  episode_reward_mean: 721.2379721240586
  episode_reward_min: 632.431694521841
  episodes_this_iter: 21
  episodes_total: 522
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.000390625064028427
      cur_lr: 0.0005000000237487257
      entropy: 1.2261481285095215
      kl: 0.007468735333532095
      policy_loss: -0.0009513133554719388
      total_loss: 8.856468200683594
      vf_explained_var: 0.20327399671077728
      vf_loss: 8.857416152954102
    grad_

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 23.2/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING [pid=361], 10117 s, 474 iter, 31994437 ts, 733 rew

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-19_00-55-43
  done: false
  episode_len_mean: 750.0
  episode_reward_max: 809.9162448933068
  episode_reward_mean: 731.8662558670126
  episode_reward_min: 637.3199588014941
  episodes_this_iter: 25
  episodes_total: 10261
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.2565784454345703
      cur_lr: 0.0005000000237487257
      entropy: 0.11439244449138641
      kl: 0.010895511135458946
      policy_loss: -0.00024692100123502314
      total_loss: 8.317546844482422
      vf_explained_var: 0.2017330825328827
      vf_loss: 8.314997673034668
   

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 23.2/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING [pid=361], 10243 s, 480 iter, 32399077 ts, 732 rew

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-19_00-57-49
  done: false
  episode_len_mean: 750.0
  episode_reward_max: 807.114498664917
  episode_reward_mean: 731.2282570174661
  episode_reward_min: 669.8361317986262
  episodes_this_iter: 22
  episodes_total: 10388
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.2565784454345703
      cur_lr: 0.0005000000237487257
      entropy: 0.09352132678031921
      kl: 0.010297712869942188
      policy_loss: 0.0005548163899220526
      total_loss: 7.117467403411865
      vf_explained_var: 0.21547037363052368
      vf_loss: 7.114270210266113
    g

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 23.3/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING [pid=361], 10369 s, 486 iter, 32803065 ts, 734 rew

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-19_00-59-55
  done: false
  episode_len_mean: 750.0
  episode_reward_max: 804.0570788190508
  episode_reward_mean: 734.6031547265588
  episode_reward_min: 669.4381874935274
  episodes_this_iter: 20
  episodes_total: 10518
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.2565784454345703
      cur_lr: 0.0005000000237487257
      entropy: 0.09570444375276566
      kl: 0.011193747632205486
      policy_loss: -0.00010967242997139692
      total_loss: 7.019583702087402
      vf_explained_var: 0.18570804595947266
      vf_loss: 7.016821384429932
  

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 23.4/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING [pid=361], 10498 s, 492 iter, 33207273 ts, 734 rew

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-19_01-02-03
  done: false
  episode_len_mean: 750.0
  episode_reward_max: 807.9285036023809
  episode_reward_mean: 732.399529665289
  episode_reward_min: 661.4798604808495
  episodes_this_iter: 24
  episodes_total: 10649
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.2565784454345703
      cur_lr: 0.0005000000237487257
      entropy: 0.08584172278642654
      kl: 0.011561415158212185
      policy_loss: -0.00043632651795633137
      total_loss: 7.630176067352295
      vf_explained_var: 0.19686436653137207
      vf_loss: 7.627645492553711
   

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 50/63 CPUs, 0/0 GPUs
Memory usage on this node: 23.4/94.8 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - PPO_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING [pid=361], 10622 s, 498 iter, 33612163 ts, 732 rew

Result for PPO_MultiWaveAttenuationMergePOEnv-v0_0:
  custom_metrics: {}
  date: 2019-04-19_01-04-09
  done: false
  episode_len_mean: 750.0
  episode_reward_max: 809.146224441628
  episode_reward_mean: 728.4292390031237
  episode_reward_min: 646.9867940586073
  episodes_this_iter: 24
  episodes_total: 10780
  experiment_id: ca1ade39f0914adcb484936affff8a1a
  hostname: flow-main
  info:
    default:
      cur_kl_coeff: 0.2565784156322479
      cur_lr: 0.0005000000237487257
      entropy: 0.10710359364748001
      kl: 0.010123347863554955
      policy_loss: 9.109894745051861e-05
      total_loss: 7.883522033691406
      vf_explained_var: 0.23232530057430267
      vf_loss: 7.880834102630615
    g